In [ ]:
import pathlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from orchid import (project_loader as opl)

In [ ]:
import toolz.curried as toolz

In [ ]:
import UnitsNet

In [ ]:
project_filenames = {
    'permian-a': 'Project-frankNstein_Permian_UTM13FT_0412_PjtDataFrame.ifrac',
    'gng': 'GnG_DemoProject_wDataFrames.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: opl.ProjectLoader(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
project_units = toolz.valmap(lambda p: p.ProjectUnits, projects)
project_units

In [ ]:
def accumulator(so_far, element):
    return toolz.assoc(so_far,  element.ObjectId, element)

def data_frame_map(project):
    return toolz.pipe(project.DataFrames.Items,
                   list,
                   lambda dfs: toolz.reduce(accumulator, dfs, {})
              )

net_data_frames = toolz.valmap(data_frame_map, projects)
net_data_frames

In [ ]:
def identify_data_frame(item):
    df_id, data_frame = item
    return {'id': data_frame.ObjectId, 'name': data_frame.Name, 'display_name': data_frame.DisplayName}

def identify_data_frames(data_frames):
    return toolz.pipe(data_frames,
                      # toolz.do(lambda df: print(df)),
                      lambda dfs: toolz.map(identify_data_frame, dfs.items()),
                      list)


identifications = toolz.valmap(identify_data_frames, net_data_frames)
identifications

In [ ]:
@toolz.curry
def add_project_name(pn, df_ids):
    return toolz.assoc(df_ids, 'project', pn)
    
    
def summarize_data_frames(item):
    pn, dfs_ids = item
    result = toolz.pipe(dfs_ids,
                        # toolz.do(lambda df_id: print(df_id)),
                        toolz.map(lambda df_ids: add_project_name(pn, df_ids)),
                        list,
                       )
    return pn, result

id_summaries = toolz.itemmap(summarize_data_frames, identifications)
id_summaries

In [ ]:
col_names = ['project', 'id', 'name', 'display_name']

In [ ]:
pd.DataFrame(data=id_summaries['permian-a'], columns=col_names)

In [ ]:
pd.DataFrame(data=id_summaries['gng'], columns=col_names)

In [ ]:
@toolz.curry
def find_data_frame_by_name(project_name, data_frames, name):
    project_data_frames = toolz.get(project_name, data_frames)
    candidates = toolz.pipe(project_data_frames,
                            toolz.valfilter(lambda df: df.Name == name),
                            lambda cs: cs.values(),
                            list,
                           )
    assert len(candidates) == 1
    return candidates[0]

In [ ]:
find_permian_data_frames = find_data_frame_by_name('permian-a', net_data_frames)
find_permian_data_frames

In [ ]:
@toolz.curry
def find_data_frames(project_name, data_frame_names):
    return toolz.pipe(data_frame_names,
                       toolz.map(find_data_frame_by_name(project_name, net_data_frames)),
                       list,
                      )

In [ ]:
permian_project_01, permian_fdi, permian_useismic = toolz.pipe(
    ['Project Data Frame 01', 'FDI Observations', 'C3-Microseismic Data Frame 01'],
    find_data_frames('permian-a'),
    toolz.map(lambda df: df.DataTable),
    list,
)
permian_project_01, permian_fdi, permian_useismic

In [ ]:
[c.ColumnName for c in permian_project_01.Columns]

In [ ]:
len(permian_project_01.Rows)

In [ ]:
# Adapted from code at
# https://docs.microsoft.com/en-us/dotnet/framework/data/adonet/dataset-datatable-dataview/creating-a-datareader
# retrieved on 18-Apr-2021.

def print_columns(reader):
    # Loop through all rows in the reader
    while reader.Read():
        for i in range(reader.FieldCount):
            print(f'{reader[i]} ', end='')
        print()
        
def print_data_table_contents(data_table):
    reader = data_table.CreateDataReader()
    try:
        is_finished = False
        while not is_finished:
            if not reader.HasRows:
                print("Empty data reader.")
                is_finished = True
            else:
                print_columns(reader)
                is_finished = reader.NextResult()
    finally:
        reader.Dispose()

In [ ]:
print_data_table_contents(permian_project_01)